In [194]:
# загружаем зависимости
!pip install --upgrade pip
!pip install sympy
!pip install numpy
!pip install scipy

In [1]:
# библиотека для символьных вычислений
from sympy import Rational, sqrt, I, symbols, simplify, Eq, latex
import numpy as np
from itertools import product

In [2]:
# задаём константы

eps = -Rational(1, 2) + sqrt(3)/2*I

# t_{ij}^{(m)} <=> t[m][i][j]
t = np.array([
    [
        [eps, 0, 0],
        [0, 1, 0],
        [0, 0, eps**2],
    ],
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ],
    [
        [eps**2, 0, 0],
        [0, 1, 0],
        [0, 0, eps],
    ],
    [
        [0, 1, 1],
        [1, 0, 1],
        [1, 1, 0],
    ],
    [
        [0, -1, 1],
        [1, 0, -1],
        [-1, 1, 0],
    ],
    [
        [0, eps, 1],
        [eps, 0, eps**2],
        [1, eps**2, 0],
    ],
    [
        [0, eps**2, 1],
        [eps**2, 0, eps],
        [1, eps, 0],
    ],
    [
        [0, -eps**2, 1],
        [eps**2, 0, -eps],
        [-1, eps, 0],
    ],
    [
        [0, -eps, 1],
        [eps, 0, -eps**2],
        [-1, eps**2, 0],
    ],
])

t[4] *= I / sqrt(3)
t[7] *= I / sqrt(3)
t[8] *= I / sqrt(3)

# h_{d,m}
h = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1],  # phi1
    [1, 1, 1, 1, -1, 1, 1, -1, -1],  # phi2
    [eps, 1, eps**2, 1, -1, eps**2, eps, -eps, -eps**2],  # phi3
    [eps**2, 1, eps, 1, -1, eps, eps**2, -eps**2, -eps],  # phi4
    [eps**2, 1, eps, 1, 1, eps, eps**2, eps**2, eps],  # phi5
    [eps, 1, eps**2, 1, 1, eps**2, eps, eps, eps**2],  # phi6
])

In [3]:
# задаём перестановки

# psi: (0, 1, 2, 3, 4, 5, 6, 7, 8)
def psi_even(m):
    return m

# psi: (2, 1, 0, 3, 4, 6, 5, 8, 7)


def psi_odd(m):
    match m:
        case 0:
            return 2
        case 2:
            return 0
        case 5:
            return 6
        case 6:
            return 5
        case 7:
            return 8
        case 8:
            return 7
        case _:
            return m


psi = np.array([
    psi_even,
    psi_odd,
    psi_odd,
    psi_odd,
    psi_even,
    psi_even,
])

In [4]:
# 54 переменные искомой системы
X = symbols("x:9")
Y = symbols("y:9")
Z = symbols("z:9")
A = symbols("a:9")
B = symbols("b:9")
C = symbols("c:9")

In [5]:
# phi_d(x_m)
def phi(d, X, m):
    return h[d-1][m]*X[psi[d-1](m)]


# правая часть уравнения из исходной системы
def f(i, j, k, l, r, s):
    if (i == j and k == l and r == s) and not (i == j == k == l == r == s):
        return 1
    if (j == k and l == r and s == i) and not (i == j == k == l == r == s):
        return -1
    return 0

In [6]:
# левая часть уравнения
def generate_left(m, p, q):
    left = 2*(X[m]*Y[p]*Z[q] + X[p]*Y[q]*Z[m] + X[q]*Y[m]*Z[p] +
              phi(2, X, m)*phi(2, Y, p)*phi(2, Z, q) +
              phi(2, X, p)*phi(2, Y, q)*phi(2, Z, m) +
              phi(2, X, q)*phi(2, Y, m)*phi(2, Z, p))
    for d in range(3, 7):
        left += phi(d, A, m)*phi(d, B, p)*phi(d, C, q) +\
            phi(d, A, p)*phi(d, B, q)*phi(d, C, m) +\
            phi(d, A, q)*phi(d, B, m)*phi(d, C, p)
    return simplify(left)


# правая часть уравнения
def generate_right(m, p, q):
    right = 0
    for i, j, k, l, r, s in product(range(3), repeat=6):
        right += t[m][i][j]*t[p][k][l]*t[q][r][s]*f(i, j, k, l, r, s)
    return simplify(right)


def generate_equation(m, p, q):
    left = generate_left(m, p, q)
    right = generate_right(m, p, q)
    return Eq(left, right)


equations = []
is_counted = np.zeros((9, 9, 9), bool)
with open("equations.tex", "w") as equations_file:
    for m, p, q in product(range(9), repeat=3):
        if is_counted[m][p][q]:
            continue

        # поскольку уравнения для шестёрок (m, p, q), (p, q, m), (q, m, p),
        # (phi_2(m), phi_2(p), phi_2(q)), (phi_2(p), phi_2(q), phi_2(m)), (phi_2(q), phi_2(m), phi_2(p))
        # оказываются идентичными
        is_counted[m][p][q] = True
        is_counted[p][q][m] = True
        is_counted[q][m][p] = True
        is_counted[psi[1](m)][psi[1](p)][psi[1](q)] = True
        is_counted[psi[1](p)][psi[1](q)][psi[1](m)] = True
        is_counted[psi[1](q)][psi[1](m)][psi[1](p)] = True

        equation = generate_equation(m, p, q)
        if equation == True:
            continue
        equations.append(equation)
        equations_file.write(f"\\[ {latex(equation)} \\]\n")

len(equations)

125

## Compressed form of equations

In [15]:
# для уравнений в сжатом виде
XYZ = symbols("xyz:9:9:9")
ABC = symbols("abc:9:9:9")

In [16]:
def get_index(m, p, q):
    return 81*m + 9*q + p


# phi_d(p_{mpq})
def phi(d, XYZ, m, p, q):
    return h[d-1][m]*h[d-1][p]*h[d-1][q]*XYZ[get_index(psi[d-1](m), psi[d-1](p), psi[d-1](q))]

In [17]:
# левая часть уравнения
def generate_left(m, p, q):
    left = 2*(XYZ[get_index(m, p, q)] + phi(2, XYZ, m, p, q))
    for d in range(3, 7):
        left += phi(d, ABC, m, p, q)
    return simplify(left)


compressed_equations = []
is_counted = np.zeros((9, 9, 9), bool)
with open("compressed_equations.tex", "w") as equations_file:
    for m, p, q in product(range(9), repeat=3):
        if is_counted[m][p][q]:
            continue

        # поскольку уравнения для шестёрок (m, p, q), (p, q, m), (q, m, p),
        # (phi_2(m), phi_2(p), phi_2(q)), (phi_2(p), phi_2(q), phi_2(m)), (phi_2(q), phi_2(m), phi_2(p))
        # оказываются идентичными
        is_counted[m][p][q] = True
        is_counted[p][q][m] = True
        is_counted[q][m][p] = True
        is_counted[psi[1](m)][psi[1](p)][psi[1](q)] = True
        is_counted[psi[1](p)][psi[1](q)][psi[1](m)] = True
        is_counted[psi[1](q)][psi[1](m)][psi[1](p)] = True

        equation = generate_equation(m, p, q)
        if equation == True:
            continue
        compressed_equations.append(equation)
        equations_file.write(f"\\[ {latex(equation)} \\]\n")

## Task 1

In [7]:
# переменные, относительно которых решаем систему
vars = [X[7], Y[7], Z[7], A[7], B[7], C[7], X[8], Y[8], Z[8], A[8], B[8], C[8]]

def contains(equation, vars):
    return set(vars) & equation.free_symbols

certain_equations = []
for equation in equations:
    if contains(equation, vars):
        certain_equations.append(equation)

len(certain_equations)

65

In [8]:
# Значения, найденные Козловым Александром
#
# X:    0.00329973  -0.00262951 | -0.00525902  -0.0032892  0.496904 |     0.501034
# Y:      -15293.6     -50.4355 |     50.5948    -30587.2 -0.512621 | -0.000767047
# Z:    0.00330954   0.00264041 |  0.00528083 -0.00329919  0.503107 |    -0.498967
# A:      -30587.1     -100.866 |     101.186    -61174.4  -1.48738 |  0.000661904
# B:    0.00330954   0.00264041 |  0.00528083 -0.00329919  0.503107 |    -0.498967
# C:    0.00329973  -0.00262951 | -0.00525904  -0.0032892  0.496904 |     0.501034

# (0) (2) | (5) (6) (3) | (4)

subs = [
    (X[0], 0.00329973), (X[2], -0.00262951), (X[5], -0.00525902), (X[6], -0.0032892), (X[3], 0.496904), (X[4], 0.501034),
    (Y[0], -15293.6), (Y[2], -50.4355), (Y[5], 50.5948), (Y[6], -30587.2), (Y[3], -0.512621), (Y[4], -0.000767047),
    (Z[0], 0.00330954), (Z[2], 0.00264041), (Z[5], 0.00528083), (Z[6], -0.00329919), (Z[3], 0.503107), (Z[4], -0.498967),
    (A[0], -30587.1), (A[2], -100.866), (A[5], 101.186), (A[6], -61174.4), (A[3], -1.48738), (A[4], 0.000661904),
    (B[0], 0.00330954), (B[2], 0.00264041), (B[5], 0.00528083), (B[6], -0.00329919), (B[3], 0.503107), (B[4], -0.498967),
    (C[0], 0.00329973), (C[2], -0.00262951), (C[5], -0.00525904), (C[6], -0.0032892), (C[3], 0.496904), (C[4], 0.501034),
]

# подставляем найденные значения
for i, equation in enumerate(certain_equations):
    certain_equations[i] = simplify(equation.subs(subs))

### Solving the equations

In [9]:
from sympy import lambdify

expressions = [equation.lhs - equation.rhs for equation in certain_equations]
lam_certain_equations = lambdify(vars, expressions)

lam = lambda vars: lam_certain_equations(*vars)

In [10]:
from scipy.optimize import root

x0 = [0]*12

solution = root(lam, x0, method='lm')

NameError: name 'a1' is not defined

In [241]:
sum([fun**2 for fun in solution.fun])**.5

13.05348216221358

In [194]:
solution.message

'Both actual and predicted relative reductions in the sum of squares\n  are at most 0.000001'

In [195]:
solution.fun

array([ 1.58048882e-02,  1.27938456e-06,  4.20654815e-05, -4.28189917e-05,
       -1.71843350e-04, -2.89048646e+00,  1.12683811e-04, -1.90488893e+00,
        8.37708272e-05,  8.41689873e-05, -1.39220417e-01, -1.56120328e-04,
        1.67757438e-04, -1.12048719e-04, -8.38959471e-05, -1.50149255e-01,
        9.87285087e-02,  5.52245225e-05, -2.86214284e+00,  1.94207384e+00,
       -8.19467719e-05,  1.57420742e-04, -5.45406014e-05, -3.55130691e-07,
        1.28895332e-05, -7.96293770e-04,  1.67045079e+00,  9.95446259e-05,
        8.09568564e-04,  1.67897699e+00, -1.01618883e-04,  5.50429919e-06,
        9.99588516e-01, -1.29083156e-05,  7.33613881e-01, -4.30522280e-05,
        7.16762684e-01,  4.29202399e-05, -1.67128652e-06, -9.99880184e-01,
       -2.49373941e-06, -2.61631599e-02,  1.63783669e-04, -1.64275431e-04,
       -8.65239820e-07, -1.08813043e-04, -1.09657676e-04, -1.97532935e-01,
       -5.56672411e-03, -1.72435617e-07])

In [218]:
solution.x[:12]

array([-3.40658010e-03, -6.07660737e-06, -1.62232981e+02, -2.03875851e+01,
        4.00381765e-03, -3.89926229e-03, -4.06417581e+02, -2.08740888e+02,
        3.38358834e-03,  8.44993133e-06, -3.59686266e-03,  4.43098014e-06])

### Check linear equations

In [11]:
# отбираем линейные уравнения

linear_equations = []
for equation in certain_equations:
    if equation.as_poly(vars).is_linear:
        linear_equations.append(equation)

len(linear_equations)

49

In [12]:
# решаем полученную систему уравнений

from sympy import linsolve

result = linsolve(linear_equations, vars)

result

EmptySet

In [19]:
from sympy import linear_eq_to_matrix

M, b = linear_eq_to_matrix(linear_equations, vars)

M.col_insert(12, b).upper_triangular()

Matrix([
[-101.229561888,  2.18411768484e-5, -100.929501456, 2.18411768484e-5, -201.858342966, -202.458461868, 0.26634079711,  1.38859689982e-5,    -0.26524130321,  1.38859689982e-5,    -0.53045631132,  0.53265519012,    0],
[             0, -1.38859689982e-5,  0.26524130321,  6.9429844991e-6, -0.26522815566,  0.26632759506, 101.229561888, -2.18411768484e-5,     100.929501456,  1.09205884242e-5,    -100.929171483, -101.229230934,    0],
[             0,                 0, -0.00659946*y1,    0.00330954*c1,  0.00329973*a1,    -30587.1*b1,   -100.871*z1,     0.00528082*x1,    -0.00525902*y1,    -0.00264041*c1,     0.00262951*a1,     100.866*b1,    0],
[             0,                 0,              0,   -0.00528082*c1,  0.00525902*a1,     201.732*b1,   -30587.2*z1,     0.00661908*x1,     0.00659946*y1,     0.00661908*c1,     0.00659946*a1,    -61174.2*b1,    0],
[             0,                 0,              0,                0,  0.33283056618,   80.762484711, 0.33383660934, -1.7425280

In [20]:
# ещё раз убеждаемся, что система не имеет решения

M.gauss_jordan_solve(b)

ValueError: Linear system has no solution